In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
path_csi_hc = 'J:\\Data\\Wi-Fi_HC\\180_100\\'

# data info
df_info = pd.read_csv('data_subc_sig_v1.csv')
df_info = df_info[(df_info.id_direction==1)  &  (df_info.id_location==1)]

person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))

# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_time = 500 #int(np.max(df_info['len']))
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

# make data generator
def gen_csi(filenames,id_num,len_num):
    for file in filenames:
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)
        '''
        abs_sub = np.mean(np.abs(data1),axis=(0,2,3))
        data1_norm = data1/abs_sub[np.newaxis,:,np.newaxis,np.newaxis]

        data1_abs = np.abs(data1_norm)
        data1_ph = np.angle(data1_norm)
        '''
        data1_diff = np.abs(data1[:500,:,:,:])
        
        # differentiation
        #data1_diff = np.diff(data1_abs,axis=0)
        
        # zero pad
        pad_len = len_num - data1_diff.shape[0]
        data1_pad = np.pad(data1_diff,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)

        # Label
        id_key = df_info[df_info.id==file]['id_person'].values[0].astype('int')
        data1_y = dict_id[id_key]

        x_input = data1_pad.reshape([-1,len_num,30,6]).astype('float32') / max_value
        y_input = tf.keras.utils.to_categorical(data1_y, no_classes).reshape([1,-1]).astype('int')
        
        yield(x_input ,y_input)

C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#models
def simple_cnn(input_shape):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        input_shape=input_shape
    ))
    model.add(tf.keras.layers.Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu'
    ))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(units=256, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Dense(units=no_classes, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [3]:
#Train Test Split
tr_idx,te_idx = train_test_split(df_info.index,test_size=0.2,shuffle=False)
df_train = df_info.loc[tr_idx]
df_test = df_info.loc[te_idx]

filenames_train = np.unique(df_train.id.values)
filenames_test = np.unique(df_test.id.values)

gen_train = gen_csi(filenames_train,no_classes,csi_time)
gen_test = gen_csi(filenames_test,no_classes,csi_time)

#trining
simple_cnn_model = simple_cnn(input_shape)
simple_cnn_model.fit_generator(gen_train,steps_per_epoch=10,epochs=10)
                              #validation_data=gen_test,validation_steps=10)


Epoch 1/10
10/10 [==============================] - 10s 1s/step - loss: 0.4487 - acc: 0.9000
Epoch 2/10
10/10 [==============================] - 1s 131ms/step - loss: 14.5063 - acc: 0.1000
Epoch 3/10
10/10 [==============================] - 2s 191ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 4/10
10/10 [==============================] - 2s 219ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 5/10
10/10 [==============================] - 2s 192ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 6/10
10/10 [==============================] - 2s 198ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 7/10
10/10 [==============================] - 2s 230ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 8/10
10/10 [==============================] - 2s 231ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 9/10
10/10 [==============================] - 2s 217ms/step - loss: 16.1181 - acc: 0.0000e+00
Epoch 10/10
10/10 [==============================] - 2s 211ms/step - loss: 16.1181 - acc: 0.0000e+00


In [4]:
file = filenames_train[0]

In [5]:
df_test

,id,len,mean,std,id_person,id_location,id_direction,id_exp,id_paper,file,name,id_sign,median,max
43884,S090_1_1_2,10640.0,12.763447,9.251473,90,1,1,2,IMG_0648,IMG_0648_9.png,김명회,90,8.265471,39.527047
43885,S090_1_1_3,10687.0,12.760390,9.233542,90,1,1,3,IMG_0648,IMG_0648_10.png,김명회,90,8.325632,38.985455
43886,S090_1_1_3,10687.0,12.760390,9.233542,90,1,1,3,IMG_0648,IMG_0648_2.png,김명회,90,8.325632,38.985455
43887,S090_1_1_3,10687.0,12.760390,9.233542,90,1,1,3,IMG_0648,IMG_0648_3.png,김명회,90,8.325632,38.985455
43888,S090_1_1_3,10687.0,12.760390,9.233542,90,1,1,3,IMG_0648,IMG_0648_4.png,김명회,90,8.325632,38.985455
43889,S090_1_1_3,10687.0,12.760390,9.233542,90,1,1,3,IMG_0648,IMG_0648_5.png,김명회,90,8.325632,38.985455
43890,S090_1_1_3,10687.0,12.760390,9.233542,90,1,1,3,IMG_0648,IMG_0648_7.png,김명회,90,8.325632,38.985455
43891,S090_1_1_3,10687.0,12.760390,9.233542,90,1,1,3,IMG_0648,IMG_0648_8.png,김명회,90,8.325632,38.985455
43892,S090_1_1_3,10687.0,12.760390,9.233542,90,1,1,3,IMG_0648,IMG_0648_9.png,김명회,90,8.325632,38.985455
43893,S090_1_1_4,10831.0,12.791918,9.200354,90,1,1,4,IMG_0648,IMG_0648_10.png,김명회,90,8.265397,39.133221


In [9]:
simple_cnn_model.targets

[<tf.Tensor 'dense_1_target:0' shape=(?, ?) dtype=float32>]